![sql](images/sql-logo.jpg)

In [7]:
import pandas as pd
import sqlite3
#import pandasql





conn = sqlite3.connect('data/flights.db')
cur = conn.cursor()

# Objectives

- Use SQL aggregation functions with GROUP BY
- Use HAVING for group filtering
- Use SQL JOIN to combine tables using keys

# Aggregating Functions

>  A SQL **aggregating function** takes in many values and returns one value.

We have already seen some SQL aggregating functions like `COUNT()`. There are also others, like SUM(), AVG(), MIN(), and MAX().

## Example Simple Aggregations

In [8]:
# Max value for longitude
pd.read_sql('''
    
SELECT MAX(CAST(longitude as real)) AS max_longitude
FROM airports
    
''', conn)

,max_longitude
0,179.951


In [ ]:
# Max value for id in table
pd.read_sql('''
SELECT 
    MAX(CAST(id AS integer))
FROM 
    airports
''', conn)

In [ ]:
# Effectively counts all the inactive airlines 
pd.read_sql('''
    SELECT 
        COUNT()
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

We can also give aliases to our aggregations:

In [ ]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

# Grouping in SQL

We can go deeper and use aggregation functions on _groups_ using the `GROUP BY` clause.

The `GROUP BY` clause will group one or more columns together with the same values as one group to perform aggregation functions on.

## Example `GROUP BY`  Statements

Let's say we want to know how many active and non-active airlines there are.

### Without `GROUP BY`

Let's first start with just seeing how many airlines there are:

In [ ]:
df_results = pd.read_sql('''

SELECT COUNT() AS num_of_airlines
FROM airlines
''', conn)

df_results

One way for us to get the counts for each is to create two queries that will filter each kind of airline (active vs non-active) and count those values:

In [9]:
df_active = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

df_inactive = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_inactive_airlines
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

display(df_active)
display(df_inactive)

,number_of_active_airlines
0,1161


,number_of_inactive_airlines
0,4886


This works but it's inefficient.

### With `GROUP BY`

Instead, we can tell the SQL server to do the work for us by grouping values we care about for us!

In [ ]:
df_results = pd.read_sql('''

SELECT COUNT() as num_of_airlines, active
FROM airlines
GROUP BY active

''', conn)

df_results

This is great! And if you look closely, you can observe we have _three_ different groups instead of our expected two!

## Group Task

- Which countries have the highest numbers of active airlines? Return the top 10.

In [ ]:
# Note that we have a 'country' field/column
pd.read_sql('''
    SELECT
        *
    FROM
        airlines
''', conn)

In [10]:
pd.read_sql("""
SELECT country, COUNT() AS num_airlines
            FROM airlines
            WHERE active = "Y"
            GROUP BY country
            ORDER BY num_airlines DESC
            LIMIT 10
            


""", conn)

,country,num_airlines
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34
5,Australia,26
6,China,25
7,Spain,24
8,Brazil,23
9,France,22


In [ ]:
pandas_way = pd.read_sql("""
SELECT *
FROM airlines


""", conn)

In [ ]:
pandas_way

In [ ]:
pandas_way.loc[pandas_way['active'] == 'Y']['country'].value_counts()[:10]

<details>
    <summary><b>Possible Solution</b></summary>

``` sql
    pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
    LIMIT 10
    ''', conn)```
</details>

> Note that the `GROUP BY` clause is considered _before_ the `ORDER BY` and `LIMIT` clauses, more on this later.

## Exercise: Grouping

- Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [13]:
df = pd.read_sql("""
SELECT *
            FROM airports
            """, conn)
        
        

df


,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8102,8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8,A,America/Vancouver
8103,8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8,A,America/Vancouver
8104,8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8,A,America/Vancouver
8105,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


In [16]:
# Your code here
pd.read_sql("""
            
SELECT timezone, COUNT() AS num_airports
FROM airports
GROUP BY timezone
ORDER BY num_airports DESC
""", conn)

,timezone,num_airports
0,America/New_York,628
1,America/Chicago,373
2,Europe/Berlin,319
3,America/Anchorage,258
4,Europe/Paris,232
...,...,...
289,America/Anguilla,1
290,Africa/Porto-Novo,1
291,Africa/Mbabane,1
292,Africa/Bujumbura,1


In [ ]:
pd.read_sql("""




""", conn)

<details>
    <summary><b>Possible Solution</b></summary>

``` sql
pd.read_sql('''
    SELECT 
        airports.timezone
        ,COUNT() AS num_of_airports
    FROM 
        airports
    GROUP BY
        airports.timezone
    ORDER BY
        num_of_airports DESC
''', conn) ```
</details>

# Filtering Groups with `HAVING`

We showed that you can filter tables with `WHERE`. We can similarly filter _groups/aggregations_ using `HAVING` clauses.

## Examples of Using `HAVING`

### Simple Filtering - Number of Airports in a Country

Let's come back to the aggregation of active airlines:

In [ ]:
pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
''', conn)

We can see we have a lot of results. But maybe we only want to keep the countries that have more than $30$ active airlines:

In [ ]:
pd.read_sql('''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING 
        num > 30
    ORDER BY 
        num DESC
''', conn)

## Filtering Different Aggregations - Airport Altitudes

We can also filter on other aggregations. For example, let's say we want to investigate the `airports` table.

Specifically, we want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

### Looking at the `airports` Table

In [ ]:
df_airports = pd.read_sql('''
    SELECT 
        *
    FROM 
        airports 
''', conn)

df_airports.head()

### Looking at the Highest Airport

Let's first get the highest altitude for each country:

In [ ]:
pd.read_sql('''
   
SELECT country, MAX(CAST(altitude as real)) AS max_altitude
FROM airports
GROUP BY country
ORDER BY country

   
''', conn)

### Looking at the Number of Airports Too

We can also get the number of airports for each country.

In [ ]:
pd.read_sql('''

SELECT country, MAX(CAST(altitude as real)) AS max_altitude, COUNT() AS num_airports
FROM airports
GROUP BY country
ORDER BY country

''', conn)

### Filtering on Aggregations

> Recall:
>
> We want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

In [ ]:
pd.read_sql('''

SELECT country, MAX(CAST(altitude as real)) AS max_altitude
FROM airports
GROUP BY country
HAVING COUNT() >= 100
ORDER BY country


''', conn)

# Joins

The biggest advantage in using a relational database (like we've been with SQL) is that you can create **joins**.

> By using **`JOIN`** in our query, we can connect different tables using their _relationships_ to other tables.
>
> Usually we use a key (*foreign key*) to tell us how the two tables are related.

There are different types of joins and each has their different use case. This is very similair to pandas `.join(), .merge()` as well as tableau relationships.

## `INNER JOIN`

> An **inner join** will join two tables together and only keep rows if the _key is in both tables_

![](images/inner_join.png)

Example of an inner join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    INNER JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Inner Joins

Let's say we want to look at the different airplane routes

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes 
''', conn)

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines 
''', conn)

This is great but notice the `airline_id` column. It'd be nice to have some more information about the airlines associated with these routes.

We can do an **inner join** to get this information!

#### Inner Join Routes & Airline Data

In [ ]:
pd.read_sql('''

SELECT *
FROM routes AS r
    INNER JOIN airlines AS al
        ON r.airline_id = al.id

''', conn)

We can also specify that we want to retain only certain columns in the `SELECT` clause:

In [ ]:
pd.read_sql('''

SELECT r.airline AS r_airline, r.source AS departing, r.dest AS destination, r.stops, al.name AS airline
FROM routes AS r
    INNER JOIN airlines AS al
        ON r.airline_id = al.id
''', conn)

#### Note: Losing Data with Inner Joins

Since data rows are kept only if _both_ tables have the key, some data can be lost

In [ ]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

df_routes_after_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

In [ ]:
# Look at how the number of rows are different
df_all_routes.shape, df_routes_after_join.shape

If you want to keep your data from at least one of your tables, you should use a left join instead of an inner join.

## `LEFT JOIN`

> A **left join** will join two tables together and but will keep all data from the first (left) table using the key provided.

![](images/left_join.png)

Example of a left and right join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    LEFT JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Left Join

Recall our example using an inner join and how it lost some data since the key wasn't in both the `routes` _and_ `airlines` tables. 

In [ ]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

# This will lose some data (some routes not included)
df_routes_after_inner_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

# The number of rows are different
df_all_routes.shape, df_routes_after_inner_join.shape

If wanted to ensure we always had every route even if the key in `airlines` was not found, we could replace our `INNER JOIN` with a `LEFT JOIN`:

In [ ]:
# This will include all the data from routes
df_routes_after_left_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        LEFT JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

df_all_routes.shape, df_routes_after_left_join.shape

In [ ]:
df_all_routes.isna().sum()

In [ ]:
df_routes_after_left_join.isna().sum()

SQLlite will default to an **INNER JOIN** if not specified

## Exercise: Joins

Which airline has the most routes listed in our database?

In [ ]:
# Your code here
pd.read_sql("""



""", conn)

<details>
    <summary><b>Possible Solution</b></summary>

```sql
SELECT
    airlines.name AS airline,
    COUNT() AS number_of_routes
-- We first need to get all the relevant info via a join
FROM
    routes
    -- LEFT JOIN since we want all routes (even if airline id is unknown)
    LEFT JOIN airlines
        ON routes.airline_id = airlines.id
-- We need to group by airline's ID
GROUP BY
    airlines.id
ORDER BY
    number_of_routes DESC
```
</details>

# Level Up: Execution Order

```SQL
SELECT 
    COUNT(table2.col2) AS my_new_count
    ,table1.col2
FROM
    table1
    JOIN table2
        ON table1.col1 = table2.col2
WHERE
    table1.col1 > 0
GROUP BY
    table2.col1
```

1. `From`
2. `Where`
3. `Group By`
4. `Having`
5. `Select`
6. `Order By`
7. `Limit`

In [ ]:
pd.read_sql("""

SELECT
FROM 
WHERE 
GROUP BY
HAVING 
ORDER BY
LIMIT 

""")

In [ ]:
# DOES NOT LIKE THIS AT ALL....
pd.read_sql("""

FROM routes AS r
    LEFT JOIN airlines AS al
        ON r.airline_id = al.id
GROUP BY r.airline_id
SELECT COUNT() AS num_routes, al.name, r.airline
ORDER BY num_routes DESC
LIMIT 1


""", conn)